In [18]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [42]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [26]:
class My_neural_network(nn.Module):
    def __init__(self):
        super(My_neural_network, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1000),
            nn.ReLU(),
            nn.Linear(1000, 100),
            nn.ReLU(),
            nn.Linear(100, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [25]:
training_data = datasets.FashionMNIST(
  root="./tutorial/data",
  train=True,
  download=False,
  transform=ToTensor()
)

test_data = datasets.FashionMNIST(
  root="./tutorial/data",
  train=False,
  download=False,
  transform=ToTensor()
)


In [36]:
def train_loop(dataloader, model, loss_fn, optimizer, batch_size):
  size = len(dataloader.dataset)

  model.train()

  for batch, (X, y) in enumerate(dataloader):
    pred = model(X) #forward
    loss = loss_fn(pred, y)

    loss.backward()
    optimizer.step()
    
    optimizer.zero_grad() #reset all gradients

    if batch % 20 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  model.eval() # unnecessary in this situation but is a good practice
  #eval() === Test()
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [40]:
def main(): 
  batch_size = 500
  learning_rate = 0.03
  epochs = 20

  train_dataloader = DataLoader(training_data, batch_size=batch_size)
  test_dataloader = DataLoader(test_data, batch_size=batch_size)

  model = My_neural_network()

  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

  for epoch in range(epochs):
    print(f"Epoch {epoch+1}\n ---------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer, batch_size)
    test_loop(test_dataloader, model, loss_fn)
  

main()


Epoch 1
 ---------------------------
loss: 2.304055  [  500/60000]
loss: 0.814735  [10500/60000]
loss: 0.678321  [20500/60000]
loss: 0.460713  [30500/60000]
loss: 0.496757  [40500/60000]
loss: 0.549582  [50500/60000]
Test Error: 
 Accuracy: 82.5%, Avg loss: 0.490101 

Epoch 2
 ---------------------------
loss: 0.390742  [  500/60000]
loss: 0.391729  [10500/60000]
loss: 0.439607  [20500/60000]
loss: 0.366113  [30500/60000]
loss: 0.430365  [40500/60000]
loss: 0.443983  [50500/60000]
Test Error: 
 Accuracy: 83.7%, Avg loss: 0.462171 

Epoch 3
 ---------------------------
loss: 0.356696  [  500/60000]
loss: 0.364639  [10500/60000]
loss: 0.384052  [20500/60000]
loss: 0.326943  [30500/60000]
loss: 0.374211  [40500/60000]
loss: 0.401074  [50500/60000]
Test Error: 
 Accuracy: 85.1%, Avg loss: 0.418693 

Epoch 4
 ---------------------------
loss: 0.338031  [  500/60000]
loss: 0.350865  [10500/60000]
loss: 0.383816  [20500/60000]
loss: 0.298668  [30500/60000]
loss: 0.387051  [40500/60000]
loss: 